[구글 코랩(Colab)에서 실행하기](https://colab.research.google.com/github/lovedlim/bigdata_analyst_cert/blob/main/part4/ch7/p7_type2.ipynb)

# 1. 문제정의
- 도서 참고

# 2. 라이브러리 및 데이터 불러오기

In [1]:
# 데이터 불러오기
import pandas as pd
# train = pd.read_csv("mart_train.csv")
# test = pd.read_csv("mart_test.csv")
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch7/mart_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch7/mart_test.csv")

# 3. 탐색적 데이터 분석(EDA)

In [2]:
# 데이터 크기 확인
train.shape, test.shape

((700, 10), (300, 9))

In [3]:
# 샘플 확인
train.head(3)

,branch,city,customer_type,gender,product_line,total,payment_method,rating,time_of_day,day_name
0,A,Yangon,Member,Female,Health and beauty,823457.25,Ewallet,9.1,afternoon,Saturday
1,C,Naypyitaw,Normal,Female,Electronic accessories,120330.00,Cash,9.6,morning,Friday
2,A,Yangon,Normal,Male,Home and lifestyle,510788.25,Credit card,7.4,afternoon,Sunday


In [4]:
test.head(3)

,branch,city,customer_type,gender,product_line,payment_method,rating,time_of_day,day_name
0,C,Naypyitaw,Normal,Female,Fashion accessories,Ewallet,9.6,afternoon,Thursday
1,B,Mandalay,Normal,Male,Food and beverages,Credit card,4.3,evening,Wednesday
2,B,Mandalay,Member,Female,Fashion accessories,Credit card,5.0,evening,Wednesday


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   branch          700 non-null    object 
 1   city            700 non-null    object 
 2   customer_type   700 non-null    object 
 3   gender          700 non-null    object 
 4   product_line    700 non-null    object 
 5   total           700 non-null    float64
 6   payment_method  700 non-null    object 
 7   rating          700 non-null    float64
 8   time_of_day     700 non-null    object 
 9   day_name        700 non-null    object 
dtypes: float64(2), object(8)
memory usage: 54.8+ KB


In [6]:
# 기초 통계값 확인
train.describe()

,total,rating
count,7.000000e+02,700.000000
mean,4.850780e+05,7.003429
std,3.643907e+05,1.713078
min,1.904175e+04,4.000000
25%,2.001195e+05,5.500000
50%,3.818745e+05,7.000000
75%,7.061276e+05,8.425000
max,1.563975e+06,10.000000


In [7]:
train.describe(include='object')

,branch,city,customer_type,gender,product_line,payment_method,time_of_day,day_name
count,700,700,700,700,700,700,700,700
unique,3,3,2,2,6,3,3,7
top,A,Yangon,Normal,Male,Sports and travel,Cash,evening,Saturday
freq,236,236,354,356,127,246,309,114


In [8]:
test.describe(include='object')

,branch,city,customer_type,gender,product_line,payment_method,time_of_day,day_name
count,300,300,300,300,300,300,300,300
unique,3,3,2,2,6,3,3,7
top,A,Yangon,Member,Female,Electronic accessories,Ewallet,evening,Saturday
freq,104,104,155,157,58,110,123,50


In [9]:
train.isnull().sum().sum()

0

In [10]:
test.isnull().sum().sum()

0

In [11]:
train['total'].value_counts()

,count
total,
283641.75,2
263875.50,2
415422.00,2
326450.25,2
130851.00,2
823457.25,1
33988.50,1
35626.50,1
404302.50,1


In [12]:
pd.set_option('display.float_format', '{:.10f}'.format)
train['total'].describe()

,total
count,700.0000000000
mean,485078.0175000000
std,364390.7265411940
min,19041.7500000000
25%,200119.5000000000
50%,381874.5000000000
75%,706127.6250000000
max,1563975.0000000002


# 4. 데이터 전처리

In [13]:
target = train.pop('total')

In [14]:
print(train.shape, test.shape)
train = pd.get_dummies(train)
test = pd.get_dummies(test)
print(train.shape, test.shape)

(700, 9) (300, 9)
(700, 30) (300, 30)


# 5. 검증 데이터 분할

In [15]:
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size=0.2, random_state=0)
X_tr.shape, X_val.shape, y_tr.shape, y_val.shape

((560, 30), (140, 30), (560,), (140,))

# 6. 머신러닝 학습 및 평가

In [16]:
from sklearn.metrics import mean_squared_error
def rmse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred)**0.5

In [17]:
# 선형회귀
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_tr, y_tr)
pred = lr.predict(X_val)
rmse(y_val, pred)

374016.3661648522

In [18]:
# 랜덤포레스트
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=0)
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)
rmse(y_val, pred)

385935.56337360526

In [19]:
# Xgboost
import xgboost as xgb
xg = xgb.XGBRegressor(random_state=0)
xg.fit(X_tr, y_tr)
pred = xg.predict(X_val)
rmse(y_val, pred)

442570.7070483747

In [20]:
# LightGBM
import lightgbm as lgb
lg = lgb.LGBMRegressor(random_state=0, verbose=-1)
lg.fit(X_tr, y_tr)
pred = lg.predict(X_val)
rmse(y_val, pred)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


404369.88780394377

# 7. 예측 및 결과 파일 생성

In [21]:
pred = lr.predict(test)
result = pd.DataFrame({
    'pred':pred
})
result.to_csv("result.csv", index=False)

In [22]:
pd.read_csv('result.csv')

,pred
0,485484.5740277334
1,474683.4165377701
2,486669.1574532853
3,426407.8222290426
4,599942.9042138030
...,...
295,501132.5887398274
296,554273.9221195823
297,540502.7549631427
298,550704.8821613131


# 8. 성능 개선

In [23]:
# 데이터 불러오기
import pandas as pd
# train = pd.read_csv("mart_train.csv")
# test = pd.read_csv("mart_test.csv")
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch7/mart_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch7/mart_test.csv")

# target 데이터
target = train.pop('total')

# 레이블 인코딩
from sklearn.preprocessing import LabelEncoder
cols = train.select_dtypes(include='O').columns

for col in cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])

# 검증데이터 분리
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size=0.2, random_state=0)

# 선형회귀
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_tr, y_tr)
pred = lr.predict(X_val)
print(rmse(y_val, pred))

# 최종 제출 파일
pred = lr.predict(test)
result = pd.DataFrame({
    'pred':pred
})
result.to_csv("result.csv", index=False)

# 베이스라인(원핫 인코딩): 374016.3661648522
# 레이블 인코딩: 371070.1344734549

371070.1344734549
